In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))

In [2]:
load_dotenv()
print(os.environ.get("TAVILY_API_KEY"))

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


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,api_key=os.environ.get("TAVILY_API_KEY"),include_domains = ['www.healthyschoolrecipes.com'])
search_results = search.invoke("what are the ingredients recipe for Stromboli")

In [4]:
search_results

[{'title': 'Vegetable Stromboli USDA',
  'url': 'https://healthyschoolrecipes.com/recipes/vegetable-stromboli-usda/',
  'content': 'Ingredients ; Vegetable oil, 1/4 cup ; Salt, 1 teaspoon ; Sugar, 2 tablespoons ; Dried basil, 1 tablespoons 1/4 teaspoon ; Dried oregano, 1',
  'score': 0.7636429},
 {'title': 'Stromboli USDA',
  'url': 'https://healthyschoolrecipes.com/recipes/stromboli-usda/',
  'content': 'Ingredients ; Water, warm, 3 cups ; Enriched all-purpose flour. (2 pounds, 10 ounces). 2.625 pounds ; Vegetable oil, ¼ cup ; Salt, kosher, 1',
  'score': 0.73778254}]

In [16]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Do NOT answer based on your own knowledge or assumptions."
        " ONLY respond if the tool provides a valid and relevant result."
        " If the tool fails or returns nothing useful, say you cannot find the answer."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )


In [31]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
search = TavilySearchResults(max_results=2,include_domains = ['www.indianhealthyrecipes.com/'])
tools = [search]

agent_executor = create_react_agent(model, tools,
                                    prompt=make_system_prompt(
        "You are only supposed to answer the query if the tool responds"
    ),)



In [35]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what are the ingredients recipe for palak paneer")]}
)
print(response["messages"][-1].content)

The ingredients for a traditional palak paneer recipe include:

1. Fresh spinach (Palak)
2. Paneer (Indian cottage cheese)
3. Onions
4. Tomatoes
5. Spices - Typically includes whole spices like cardamoms, cinnamon, cloves, and cumin seeds. Options to use garam masala powder.
6. Ginger garlic paste
7. Cream (optional for a restaurant touch)

For detailed preparation steps, you can refer to the [recipe source](https://www.indianhealthyrecipes.com/palak-paneer-recipe-easy-paneer-recipes-step-by-step-pics/). 

FINAL ANSWER


In [8]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=
            """what are the ingredients recipe for Stromboli""")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()



================================ Human Message =================================

what are the ingredients recipe for Stromboli
================================== Ai Message ==================================

A basic Stromboli recipe typically includes the following ingredients:

1. **Pizza Dough**: 
   - You can use homemade or store-bought pizza dough.

2. **Thinly Sliced Meats**: 
   - Common choices include pepperoni, ham, salami, or other cured meats.

3. **Cheese**:
   - Mozzarella is most common, but you can also include provolone or Parmesan.

4. **Vegetables (optional)**: 
   - Bell peppers, onions, mushrooms, or spinach can be added according to preference.

5. **Sauce**:
   - While traditional Stromboli doesn’t include sauce inside (the sauce is usually served on the side for dipping), you might include a thin layer of tomato sauce if you prefer.

6. **Herbs and Spices**:
   - Oregano, basil, garlic powder, or Italian seasoning for extra flavor.

The preparation generally i

In [12]:
tools = {
    "Click": "click",
    "Type": "type_text",
    "Scroll": "scroll",
    "Wait": "wait",
    "GoBack": "go_back",
    "Google": "to_google",
}


for node_name, tool in tools.items():
      print(f"{node_name," ",tool}" )

('Click', ' ', 'click')
('Type', ' ', 'type_text')
('Scroll', ' ', 'scroll')
('Wait', ' ', 'wait')
('GoBack', ' ', 'go_back')
('Google', ' ', 'to_google')
